In [2]:
import pandas as pd
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

# Load the data
file_path = 'D:\\Work\\Home-2023\\may-project\\labs\\outputsx\\merged\\merged_บ้านสบป้าด.csv.csv'
data = pd.read_csv(file_path)

# Convert 'Datetime' to datetime object
data['Datetime'] = pd.to_datetime(data['Datetime'])

# Optionally, extract more time-related features (e.g., hour, weekday) if needed
# data['Hour'] = data['Datetime'].dt.hour
# data['Weekday'] = data['Datetime'].dt.weekday

# Select columns that start with "ValueLn", excluding 'ValueLn_SO2'
selected_columns = [col for col in data.columns if col.startswith("ValueLn") and col != 'ValueLn_SO2']
features_data = data[selected_columns]

print(selected_columns  )

# Handling missing values
features_data = features_data.fillna(features_data.median())

# Scaling numerical features
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(features_data)

# Define the target variable
y = data['ValueLn_SO2']
y = y.fillna(y.median())

print(y.isnull().sum())


# Feature Selection
k = 5  # Number of top features to select
selector = SelectKBest(f_classif, k=k)
X_new = selector.fit_transform(scaled_data, y)

# Splitting the dataset for training and testing
X_train, X_test, y_train, y_test = train_test_split(X_new, y, test_size=0.2, random_state=42)

# X_train and X_test now contain the selected features for your LSTM model
print(list(features_data.columns[list(selector.get_support(indices=True))]))
print(selector.pvalues_)

['ValueLn_BP', 'ValueLn_DEV', 'ValueLn_NO2', 'ValueLn_PM10', 'ValueLn_RAIN', 'ValueLn_RH', 'ValueLn_SO2_1', 'ValueLn_SW', 'ValueLn_TEMP', 'ValueLn_TSP', 'ValueLn_WD', 'ValueLn_WS']
0
['ValueLn_NO2', 'ValueLn_PM10', 'ValueLn_RH', 'ValueLn_TEMP', 'ValueLn_TSP']
[9.66982510e-020 2.00907342e-088 0.00000000e+000 1.53127705e-203
 1.42194828e-006 0.00000000e+000 1.00000000e+000 3.41226041e-070
 2.33401512e-262 2.83304013e-186 5.05810931e-079 3.41226041e-070]


In [13]:
import pandas as pd
import tensorflow as tf
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.feature_selection import SelectKBest, SequentialFeatureSelector, f_classif
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import TimeSeriesSplit


# GPU configuration (optional, for fine-tuning performance)
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

# Load the data
file_path = 'D:\\Work\\Home-2023\\may-project\\labs\\outputsx\\merged\\merged_บ้านสบป้าด.csv.csv'
data = pd.read_csv(file_path)

# Convert 'Datetime' to datetime object
data['Datetime'] = pd.to_datetime(data['Datetime'])

# Select columns that start with "ValueLn", excluding 'ValueLn_SO2'
selected_columns = [col for col in data.columns if col.startswith("ValueLn") and col != 'ValueLn_SO2']
features_data = data[selected_columns]

# Handling missing values
features_data = features_data.fillna(features_data.median())

# Scaling numerical features
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(features_data)

# Define the target variable
y = data['ValueLn_SO2']
y = y.fillna(y.median())

# Feature Selection
k = 5  # Number of top features to select
selector = SelectKBest(f_classif, k=k)
X_new = selector.fit_transform(scaled_data, y)

# TimeSeriesSplit
tscv = TimeSeriesSplit(n_splits=5)
for train_index, test_index in tscv.split(X_new):
    X_train, X_test = X_new[train_index], X_new[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    print(X_train.shape, y_train.shape)
    print(X_train[0], y_train.shape)

    # Here, you can add your model training and evaluation code
    # For example:
    # model.fit(X_train, y_train)
    # Evaluate the model on X_test, y_test

# Note: The exact way you train and evaluate your model may vary depending on the model you are using


    # Build your LSTM model

# Rest of your model code
    model = Sequential()

    model.add(LSTM(50, input_shape=(X_train.shape[1], 5), activation='relu', recurrent_activation='sigmoid', return_sequences=True))
    model.add(LSTM(50, activation='relu', recurrent_activation='sigmoid'))

    model.add(Dense(1))

    model.compile(loss='mean_squared_error', optimizer='adam')

    # Train the model
    model.fit(X_train, y_train, epochs=20, batch_size=7200, validation_data=(X_test, y_test), verbose=2, shuffle=False)



    # Evaluate the model
    mse = model.evaluate(X_test, y_test, verbose=0)
    print(f'MSE for fold: {mse}')


(17532, 5) (17532,)
[0.01587302 0.04241071 0.92929293 0.39086409 0.03552398] (17532,)
Epoch 1/20


ValueError: in user code:

    File "d:\Users\kandation\me\anaconda3\envs\project-may\lib\site-packages\keras\engine\training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "d:\Users\kandation\me\anaconda3\envs\project-may\lib\site-packages\keras\engine\training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "d:\Users\kandation\me\anaconda3\envs\project-may\lib\site-packages\keras\engine\training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "d:\Users\kandation\me\anaconda3\envs\project-may\lib\site-packages\keras\engine\training.py", line 993, in train_step
        y_pred = self(x, training=True)
    File "d:\Users\kandation\me\anaconda3\envs\project-may\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "d:\Users\kandation\me\anaconda3\envs\project

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]
[  0  10  20  30  40  50  60  70  80  90 100 110 120 130 140 150 160 170
 180 190]
--------------------------------------------------
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35]
[  0  10  20  30  40  50  60  70  80  90 100 110 120 130 140 150 160 170
 180 190 200 210 220 230 240 250 260 270 280 290 300 310 320 330 340 350]
--------------------------------------------------
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51]
[  0  10  20  30  40  50  60  70  80  90 100 110 120 130 140 150 160 170
 180 190 200 210 220 230 240 250 260 270 280 290 300 310 320 330 340 350
 360 370 380 390 400 410 420 430 440 450 460 470 480 490 500 510]
--------------------------------------------------
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23